# Shallow Network
## Rede neural com uma unica camada para o problema de classificação de digitos utilizando o framework TensorFlow
## A base de dados é a MNIST com 10 classes (dígitos de 0 a 9)

In [2]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data as mnist_data

In [3]:
tf.GraphKeys.VARIABLES = tf.GraphKeys.GLOBAL_VARIABLES

(1) Definir a Arquitetura da rede:
    - Camada de entrada = número de features = número de pixels da imagem
      As imagens da MNIST tem 28x28 pixels, ou seja 784 features
    - Camada de saída = número de classes = número de dígitos
      Na MNIST temos 10 dígitos, de 0 a 9, ou seja 10 neurônios de saída
     

In [4]:
# placeholder : alocacao de tamanho indefinido
# variable : alocacao de tamanho pre-definido

# matriz de entrada
# placeholder do tipo float, com numero indefinido de imagens porem todas de tamanho 28x28x1
X = tf.placeholder(tf.float32, [None, 28, 28, 1])

In [5]:
# matriz de pesos - 784 x 10 (784 features x 10 classes)
W = tf.Variable(tf.zeros([784,10]))
# termos bias - 10, um para cada classe
b = tf.Variable(tf.zeros([10]))

In [6]:
# camada de saida (rotulos reais)
Y = tf.placeholder(tf.float32, [None, 10])

(2) Definir como serão feitos os cálculos;
uso de multiplicação matricial! $$f(\mathbf{x}) = W\mathbf{x} + \mathbf{b}$$
inicializar os pesos e outras variaveis

In [7]:
init = tf.global_variables_initializer() # instancia inicializacao

In [8]:
# As predicoes sao dadas pela formula anterior
X1 = tf.reshape(X, [-1, 784])  # vetoriza as imagens
# Y_ = tf.nn.softmax(tf.bias_add(tf.matmul(X1, W), b)) # multiplica X1 por W, e soma b
Y_ = tf.nn.softmax( tf.matmul(X1, W) + b) # multiplica X1 por W, e soma b

(3) Funcao de custo: informa quão longe estamos da solução ideal
    Entropia cruzada
    $$\sum_{i=0}^{N} -(Y \cdot \log(\hat Y))$$

In [9]:
crossEnt = -tf.reduce_sum(Y * tf.log(Y_))

In [10]:
# alem da funcao de custo vamos calcular tambem a acuracia
# Y[i]  = [0   , 0  , 0  , 1  ,  0  , 0  , 0  , 0  , 0  , 0  ] -> vetor rotulo real
# Y_[i] = [0.1 , 0  , 0.2, 0.7,  0  , 0  , 0  , 0  , 0  , 0  ] -> vetor predito pela rede
correctPred = tf.equal( tf.argmax(Y_,1), tf.argmax(Y,1) )
accuracy = tf.reduce_mean(tf.cast(correctPred, tf.float32))

(4) Metodo de otimizacao e suas variaveis

    *Gradiente Descendente* ou versões: RMSProp, Adam, etc.
    *Taxa de Aprendizado* = 0.003
    *Batch size* = 64

In [11]:
lrate = 0.003
batchSize = 64

optMethod = tf.train.GradientDescentOptimizer(lrate)
trainProcess = optMethod.minimize(crossEnt)

(5) Rodar um experimento com a MNIST

In [12]:
mnist = mnist_data.read_data_sets("data", one_hot=True, reshape=False, validation_size=0)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting data/t10k-labels-idx1-ubyte.gz


In [14]:
# Sessao TensorFlow
sess = tf.Session() # instancia a sessao
sess.run(init) # inicializa variaveis com o objeto que criamos anteriormente (init)

# geralmente 500 ou mais iteracoes sao necessarias
iterations = 20 # alterar aqui ou rodar diversas vezes o bloco a seguir

In [15]:
for i in range(iterations):
    # carrega batch um par Imagem, Rotulo
    batX, batY = mnist.train.next_batch(batchSize)
    # passar para o tensorflow, preciso definir um dicionario
    # o dicionario contem o batch no formato Chave, Valor: Imagens, Rotulos
    # a 'chave' deve ser a mesma que definimos como placeholders para a arquitetura
    # nesse caso X e Y
    trainData = {X: batX, Y: batY}
    # executa uma iteracao (feed-forward e backpropagation)
    sess.run(trainProcess, feed_dict=trainData)
    
    loss = sess.run(crossEnt, feed_dict=trainData)
    acc  = sess.run(accuracy, feed_dict=trainData)
    print(str(i) + " Loss: " + str(loss) + " Accuracy: "+str(acc*100)+"%")

0 Loss: 126.144 Accuracy: 40.625%
1 Loss: 111.234 Accuracy: 73.4375%
2 Loss: 99.1913 Accuracy: 76.5625%
3 Loss: 97.6461 Accuracy: 79.6875%
4 Loss: 92.1918 Accuracy: 73.4375%
5 Loss: 84.0675 Accuracy: 79.6875%
6 Loss: 84.4014 Accuracy: 70.3125%
7 Loss: 81.6924 Accuracy: 73.4375%
8 Loss: 75.0833 Accuracy: 76.5625%
9 Loss: 58.308 Accuracy: 81.25%
10 Loss: 65.9334 Accuracy: 81.25%
11 Loss: 63.0058 Accuracy: 79.6875%
12 Loss: 69.2995 Accuracy: 76.5625%
13 Loss: 66.1838 Accuracy: 81.25%
14 Loss: 51.9511 Accuracy: 89.0625%
15 Loss: 53.9105 Accuracy: 87.5%
16 Loss: 52.5584 Accuracy: 85.9375%
17 Loss: 51.3248 Accuracy: 87.5%
18 Loss: 56.7684 Accuracy: 84.375%
19 Loss: 50.7153 Accuracy: 82.8125%


(6) Avaliar o modelo em dados nao vistos no treinamento

In [16]:
testData = {X: mnist.test.images, Y: mnist.test.labels}
accTest  = sess.run(accuracy, feed_dict=testData)
print("Accuracy on test data: " + str(accTest*100)+"%")

Accuracy on test data: 79.5799970627%
